# Language models
A language model is an algorithm that takes a sequence of words, and outputs the likely next word in the sequence. Most language models output a list of words, each with its probability of occurance. For example, if we had a sentence that started `I would like to eat a hot`, then ideally the algorithm would predict that  the word `dog` had a much higher chance of being the next word than the word `meeting`. Many of you may be familiar with some aspects of this concept from CS136's WordGen lab, or CS134's Generators lab assignment, although, in that case you were likely using characters rather than entire words in your language models.

Language models are a very powerful building block in natural language processing. They are used for classifying text (e.g. is this review positive or negative?), for answering questions based on text (e.g. "what is the capital of Finland?" based on the Wikipedia page on Finland), and language translation (e.g. English to Japanese).

## The intuition behind why language models are so broadly useful
How can this simple sounding algorithm be that broadly useful? Intuitively, this is because predicting the next word in a sentence requires a lot of information, not just about grammar and syntax, but also about semantics: what things mean in the real-world. For instance, we know that `I would like to eat a hot dog` is semantically reasonable, but `I would like to eat a hot cat` is nonsensical. 

I trained a simple language model, and asked it to predict the word following `I would like to eat a `. We get: `hamburger`

The rest of this notebook will describe how to set up a language model using python modules available to us, to make such word predictions as well as classifications. We will use these modules to classify and generate text in an automated chatbot on Twitter and then reflect on the process, bringing together all our class discussions about conversational agents, natural language technologies, and human-in-the-loop systems!
    

# Step 0: Set-up for this Assignment

You'll need to install some new modules to complete this assignment. Set-up instructions are always available via the project README in the Github repository.

Once you've done so, you should be able to run the following code:

In [13]:
# Follow install instructions in the README, prior to running this notebook!

# If using Google Colab, uncomment these two lines:
#!pip install fastai==2.0.19
#!pip install fastcore==1.3.2

from fastai.text import *
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Step 1: Load all the data 
In this activity, we are going to use a dataset of tweets from the satirical news site, [The Onion](https://www.theonion.com), as well as some non-sarcastic news sources. The data set is from [Kaggle](https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection). 

The dataset is available as a JSON file in the included folder, `lib`.


In [14]:
from pathlib import Path
data_path = Path('./lib')

The data is in a JSON file, so we are using the `read_json` method to read-in the data. If you have different data that is CSV, use the `read_csv` method instead. 

We use the `lines=True` argument here because the author formatted each line as a separate JSON object. At least half of your time as a data scientist/AI researcher is spent dealing with other people's data formats!


In [15]:
headlines = pd.read_json('/content/drive/MyDrive/data/Sarcasm_Headlines_Dataset_v2.json', lines=True)

In [16]:
headlines

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...
...,...,...,...
28614,1,jews to celebrate rosh hashasha or something,https://www.theonion.com/jews-to-celebrate-ros...
28615,1,internal affairs investigator disappointed con...,https://local.theonion.com/internal-affairs-in...
28616,0,the most beautiful acceptance speech this week...,https://www.huffingtonpost.com/entry/andrew-ah...
28617,1,mars probe destroyed by orbiting spielberg-gat...,https://www.theonion.com/mars-probe-destroyed-...


As you can see, some of this dataset is drawn from _The Onion_, the rest is drawn from places like the Huffington Post which publishes real news, not satire. 

## Step 1a: Examine the data set (5% effort)

Before we go off adventuring, let's first see what this dataset looks like. 

### Q1.1: How large is this dataset? Is it balanced? (1% effort)

**ANSWER:** 

- we have 28619 entries in our dataset (3 columns in each entry)

- it is not balanced (we have 13634 cases where satire was detected and 14985 where it wasn't)

In [17]:
# Insert code here to check size of dataset, and how many are positive (is_sarcastic = 1) and how many negative?
# Hint: Your output will look like this:
df_yes = headlines[headlines.is_sarcastic==1]
df_no = headlines[headlines.is_sarcastic==0]
print(df_yes.is_sarcastic.value_counts())
print(df_no.is_sarcastic.value_counts())

headlines.info()

1    13634
Name: is_sarcastic, dtype: int64
0    14985
Name: is_sarcastic, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
 2   article_link  28619 non-null  object
dtypes: int64(1), object(2)
memory usage: 670.9+ KB


### Q1.2: How many words on average is each headline? (4% effort)
Longer text = more information. We want to see what the length of the headline is in order to see how much information it may have. 

_Hint:_ See https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.count.html (the the `\s` regex looks for spaces).

**ANSWER:** 
- on average, there are 10 words per headline in our df

In [18]:
# Insert code here to find the average length of headline (in words)
s = headlines['headline']
count = s.str.count('\s')+1
average = (count.sum())/(len(s.index))
print(average)

10.052552500087355


# Step 2: Build a language model that knows how to write news headlines

This is the first step of our project that will be using a machine learning model. 

We are going to use the [fast.ai](https://fast.ai/) library to create this model. If you need help with understanding this section, look at the fast.ai documentation -- it is fantastic! The steps below are modified from the [online tutorial](https://docs.fast.ai/tutorial.text.html#The-ULMFiT-approach).

In [19]:
import fastai
from fastai.text.all import *

*Note: if this import fails for you, make sure you've installed fastai first. This is not super straight-forward. Close the notebook and follow the instructions in the README.*

In [20]:
dls = TextDataLoaders.from_df(headlines, path=data_path, text_col='headline', is_lm=True, valid_pct=0)
print("This code takes ~1 minute to run on Iris' laptop, and possibly more than 10 minutes on Colab.")


This code takes ~1 minute to run on Iris' laptop, and possibly more than 10 minutes on Colab.


First, we tell fastai that we want to work on a list of texts (headlines in our case), that are stored in a dataframe (that's the `TextDataLoaders.from_df` part.) We also pass in our data path, so after we process our data, we can store it at that location. We tell it where to look for the headline in the dataframe (which  column to use, `text_col=`). We specify that it is a language model (`is_lm`) so it labels the "next word" as the label for each sequence of words. But what kind of validation data do we need for a language model? Remember that a language model predicts the next word in an input sequence of words. So, we can't just take some of the headlines and set them aside as validation data. Instead, we want to use all the sentences and validate whether we can guess the right next word some fraction of the time. So, we tell it not to split the data into validation sets (`valid_pct=0`). 

## Step 2a: Learn the model

Now that we have the data, it's time to train the model.

Now, we *could* learn a language model from scratch. But we're instead going to cheat. We're going to use a pretrained language model, and finetune it for our purpose. Specifically, we're going to use a model trained on the `Wikitext-103` corpus. 

One way to understand it is to think of our pre-trained model is as a model that can predict the next word in a Wikipedia article. We want to train it to write headlines instead. Since headlines still have to sound like English, ie. follow grammar, syntax, be generally plausible etc, being able to predict the next word in Wikipedia is super useful. It allows us to start with a model that already knows some English, and then just train it for writing headlines.



In [21]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.5)

In [22]:
#

This `AWD_LSTM` is the pretrained Wikipedia model.

Let's train it.

In [23]:
learn.fit_one_cycle(1, 1e-2)
print("This code takes Iris' laptop about 8 minutes to run, and around 15 minutes on Colab.")

epoch,train_loss,valid_loss,time
0,5.963179,None,11:23


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


This code takes Iris' laptop about 8 minutes to run, and around 15 minutes on Colab.


Once trained, it's time to write some headlines! We give it a starting sequence `Students protest ` and see what it comes up with. 

_Note:_ Don't worry about a `UserWarning: Your generator is empty.` warning, unless the next two lines of code don't work!

_Note:_ If you were unable to get the model training/building working, it _might_ be possible to load the pre-built `headlines-lm.pkl` model that's available [via Google Drive](https://drive.google.com/drive/folders/1aKO1eWeGJmHZFrslRnMQRV1PVFiPv6eB?usp=sharing) and load it with `load_learner(fname='lib/headlines-lm.pkl')`. If everything is working, don't worry about this comment!

In [24]:
learn.predict("Students protest ", n_words=5, no_unk=True)

'Students protest against not all college students'

Pretty good, huh? 

In [25]:
learn.predict('The Fed is expected to', n_words=3, no_unk=True)

'The Fed is expected to have a great'

OK, it's not perfect! Let's make it a little better. 

The `unfreeze` below is telling fastai to allow us to change the weights throughout the model. We do this when we want to make the model generate text that's more similar to our headlines (than to Wikipedia). 

In [26]:
learn.unfreeze()

In [27]:
learn.fit_one_cycle(1, 1e-3)
print("This code takes Iris' laptop about 11 minutes to run, 20 minutes on Colab.")

epoch,train_loss,valid_loss,time
0,5.157502,None,16:46


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


This code takes Iris' laptop about 11 minutes to run, 20 minutes on Colab.


In [28]:
learn.predict('New Study', n_words=5)

'New Study : to solve legal problems'

In [29]:
learn.predict('16 Problems', n_words=5)

'16 Problems of the Week :'

OK, now let's save our hard work. We'll use this later. (Pssst: why is it called an encoder? Look at the Fastai docs to find out!)

In [30]:
learn.save_encoder('headlines-awd.pkl')

Note that we also want to save the whole model, so we can reuse it in our twitter bot. 


In [31]:
learn.export('headlines-lm.pkl')

# Step 2b: See how well the language model works (15% effort)

Try generating a few more headlines. Then, answer the following questions. Wherever possible, show what code you ran, or what predictions you asked it for. 

*Suggestion: Try using punctuations, numbers, texts of different lengths, etc.*

### Q2.1: What is the effect of starting with longer strings? (5% effort)

We could start our headline generation with just one word, e.g. `learn.predict('White', n_words=9)` or with many: `learn.predict('White House Says Whistleblower Did', n_words=5)`. What is the difference you see in the kinds of headlines generated starting with shorter versus longer strings?

**ANSWER:** 
- longer sentences tend to make more sense than shorter sentences, this is probably because of the average word count per headline being around 10, so the model is better at generating headlines with a word count around that number. However, when I tried generating 20~30 words, the model started adding repetitive nonsensical words to finish the sentence : such as "hospitalhospitalhospital househousehouse". and that makes sense because we probably have very few -if any- headlines with +30 words in them

In [32]:
#learn.predict('White', n_words=9)
#learn.predict('White House Says Whistleblower Did', n_words=5)

#learn.predict('the students', n_words=10) == does not make sense 
  #the students head at restaurant 's new deep water community : capital

#learn.predict('the students were protesting because', n_words=5)
  #the students were protesting because it is big the
  # sentence cut short 
learn.predict('in 5 years', n_words=30)
  # 5 words  : the white house agrees , but it didn't right
               #  the crisis has case for a sibling
              # 5 words : the big news is an end to ' hopes
               # "the big news is" that children are killing ski
               #"what about 5" girls who would become citizens
              # "in 5 years from now" in popular call to u.s
  # 10 : "the white house agrees" , guilty buried in landmark enjoyed music gods with compliments
  # the big news is... :
    #  10 words : the big news is no russia , was adelson ! conservatives hope -
    # the big news is a prank for the better stars in america middle
   # "what about 5 years" that happens , twitter offers new one - way
   # "in 5 years from now peaceful European marriage enters chemistry and financial justice" --> wow this one makes sense
   # "in 5 years from" fishing man 's wife 's neck is broken leonardo dicaprio dies of mental illness in


   # 20 words :
    # "in 5 years" since the mini congress announces plans to expand urban soil revenge 250 , 000 whitewhitewhite househousehouse menmenmen movemovemove
      # what the hell is that end ?

"in 5 years behind bold re - election plans , the way i used his own fist to pretend i advocate a or , it would n't happen wishing there is nothing"

### Q2.2: What aspects of the task of generating headlines does our language model do well? (5% effort)
For example, does it get grammar right? Does it know genders of people or objects? Parts of speech? etc.

**ANSWER:** 

- it is good at grammar (examples below)
- generally good at generating politics stuff (especially sentences with the word poltician(s) in it)
- it knows the gender of people (when using names/'woman'/'man')
- it knows when you're asking it to finish a question (when giving enough words (~20) to get to the question mark)

In [33]:
# Code block with a handful of examples of what it does well


#learn.predict('this woman', n_words=20)
    # this woman 's role as a velvet traveler is judge michelle . she 's right , she sliders wanted — she thought
#learn.predict('a white man', n_words=20) 
    # a white man protesting lgbt immigrants under his own name this is the heart of ' thrones ' we 're excited
#learn.predict('this table', n_words=10) 
    #this table slam shot a little slaves morning to come back (what?)
#learn.predict('one way to do', n_words=15) 
    #this table slam shot a little slaves morning to come back (what?)
#learn.predict('jessica can', n_words=15) 
  #Jessica can buy and forget it during stage of her wedding marine officers will n't be
#learn.predict('What happens if ', n_words=20) 
  #What happens if you consider your role ? weak man had long napkin on the organ 's who hannity name was current


### Q2.3: What aspects of the task of generating headlines does our model do poorly? (5% effort)
What does it frequently get wrong? Why might it make these mistakes?

**ANSWER:** 
- words such as "religions" are marked as "xxunk" when trying to generate sentences with them so what follows is not related to them
  - However, words such as Islam, christianity, judaism, and God are not
  - perhaps the word religions, dieting, etc.. were not in the training data set
  - update : looked at the rest of the assignment and yes, those are unknown words to the model

- anything not politics related generally does not make sense which is reasonable considering we're getting our headlines from the Onion (see health example)


In [34]:
# Code block with a handful of examples of what it does poorly
#learn.predict('God will', n_words=15) 
  # God will challenge him in super bowl game like this a of the tibetan gods who 
#learn.predict('religions are now', n_words=15) 
  #xxunk are now ignored by what they 're content designers finds role of parallel content and other
#learn.predict('Christianity is now', n_words=15) 
  # Christianity is now common practice in the solar system new genius allows for an innovative attempt to 
  # christianity is what lol

#learn.predict('why dieting is now ', n_words=15)
  #vwhy xxunk is now mistaking a plane for it ? he acquired 2 different outfits in august late

#learn.predict('your health is ', n_words=15)
  #your health is hard to write and a little more netflix -- and it not going to happen
  #your health is not complete after egypt lands are manufacturing this agency uses your imagination to help

# Step 3: Learn a classifier to see which headlines are satire

Remember, our dataset has some stories that are satire (from _The Onion_) and others that are real. Now, we're going to train a classifier to distinguish one from the other. 

In [35]:
dls_clas = TextDataLoaders.from_df(headlines, path=data_path, text_col='headline', label_col='is_sarcastic', valid_pct=0.2, text_vocab=dls.vocab)
print("This code takes Iris' laptop about 1 minute to run, 13 minutes in Colab.")

This code takes Iris' laptop about 1 minute to run, 13 minutes in Colab.


We're using a similar DataLoaders method as we did for our language model above. Here, we specify the target column with `label_col` and we use `valid_pct=0.2` so we keep some fraction of our dataset as a validation set. There is one other trick: `text_vocab=dls.vocab` ensures that our classifier only uses words that we have in our language model -- so it never deals with words it hasn't encountered before. (Consider: why is this important?)

**See if you can work out what the other arguments are.**

In [36]:
dls_clas.show_batch()

,text,category
0,"xxbos hot wheels ranked number one toy for rolling down ramp , knocking over xxunk that send xxunk down a funnel , dropping onto teeter - xxunk that yanks on string , causing xxunk system to raise wooden block , xxunk series of twine xxunk that unwind spring , launching tennis ball across room , xxunk tire down slope until it hits power switch , xxunk table fan that blows toy ship with nail attached to it across xxunk pool , popping water balloon that fills cup , weighing down lever that forces basketball down track , xxunk xxunk on axis to rotate , allowing golf ball to roll into sideways coffee mug , which xxunk down row of xxunk books until handle catches hook attached to lever that causes wooden xxunk to slam down on serving spoon , xxunk small ball into cup attached by ribbon to lazy susan",1
1,"xxbos ' 12 years a slave , ' ' captain xxunk , ' ' american xxunk , ' ' wolf of wall street , ' ' blue xxunk , ' ' dallas buyers club , ' ' her , ' ' xxunk , ' ' before midnight , ' and ' xxunk ' all written during same continuing education screenwriting class",1
2,"xxbos report : doing your part to stop climate change now requires planting 30 , xxrep 3 0 new trees , getting 40 , xxrep 3 0 cars off the road , xxunk 20 square miles of coral reef",1
3,xxbos theater : xxunk is … wait for it … epic in ' xxunk ; ' daniel xxunk is impressive in ' xxunk ; ' ' the great xxunk ' is n't,0
4,xxbos roy moore on pedophilia accusers : ' these women are only xxunk me now because xxunk xxunk norms have created an environment in which assault allegations are taken seriously ',1
5,"xxbos ' men are not xxunk , ' says woman who has no idea what it like to take two whole xxunk to get to your clothing section at zara",1
6,"xxbos can america be saved ? : how did ' yes , we can ! ' become ' no , we could n't ' ? ( part one )",0
7,"xxbos ' i 'm not really looking to date right now , ' says man , as if he not at mercy of love 's powerful , mysterious ways",1
8,"xxbos something to vote for on november 8 , 2016 : elect xxunk candidates on election day and the united states leads the world in fighting climate change !",0


Above: what our data looks like after we apply the vocabulary restriction. `xxunk` is an unknown word. `xxpad` is a padding character used to ensure all strings are the same length. 

Below: we're creating a classifier:

In [37]:
classify = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

Remember that language model we saved earlier? It's time load it back!

In [38]:
classify.load_encoder('headlines-awd.pkl')

What's happening here? 

Here's the trick: a language model predicts the next word in a sequence using all the information it has so far (all the previous words). When we train a classifier, we ask it to predict the label (satire or not) instead of the next word. 

The intuition here is that if you can tell what the next word in a sentence is, you can tell if it is satirical. (Similarly, if you can can tell what the next word in an email is, you can tell if it is spam, etc.)

In [39]:
classify.fit_one_cycle(1, 1e-2)
print("This line takes Iris' laptop about 3 minutes to run, Colab 6 minutes.")

epoch,train_loss,valid_loss,accuracy,time
0,0.441844,0.373931,0.835576,05:22


This line takes Iris' laptop about 3 minutes to run, Colab 6 minutes.


In [40]:
classify.freeze_to(-2)

Above: this is similar to `unfreeze()` that we used before. Except, you only allow a few layers of your model to change. Then we can train again, similar to using `unfreeze()`

In [41]:
classify.fit_one_cycle(1, 1e-3)
print("This line takes Iris' laptop about 5 minutes to run, Colab 7 minutes.")

epoch,train_loss,valid_loss,accuracy,time
0,0.360781,0.321405,0.862135,06:10


This line takes Iris' laptop about 5 minutes to run, Colab 7 minutes.


Wow! An accuracy of >85%! (_Consider: Where is accuracy reported?_) That sounds great, and for not that much work. 

Now, let's try it on some headlines, to see how well it does. 

# Step 4: Try out the classifier (20% effort)

In [42]:
classify.predict("Despair for Many and Silver Linings for Some in California Wildfires")

('0', tensor(0), tensor([0.8573, 0.1427]))

Here in the output, the first part of this tuple is the chosen category (`0`, i.e. not satire). The second part of the result is the index of "0" in our data vocabulary, and the last part is the probabilities attributed to each class (98.1% for `0` and 0.02% for `1`). The classifier suggests that the headline (which I got from the [New York Times](https://www.nytimes.com/2019/10/29/us/california-fires-homes.html?action=click&module=Top%20Stories&pgtype=Homepage)) is not satire and it seems pretty confident that's the case (98.1% probability). 

## Step 4a: Try out this classifier (10% effort)

Below, try the classifier with some headlines, real or made up (including made up by the language model above), and identify examples it classifies (in)correctly. As you do so, you should think about what the model is doing behind-the-scenes 


### Q4.1: Two headlines that the classifier _correctly_ classifies 

**ANSWER:** 

1. "CEOs Explain Why Their Brands Stopped Advertising On Elon Musk’s Twitter"
2. "Man Embraces Holiday Spirit By Telling People He Does not Speak To His Family"

### Q4.2: Two headlines that the classifier classifies _incorrectly_

**ANSWER:** 

1. "Lucky Old Woman Getting Wheeled Around The Airport" (classifies as not satire)
2. "New President Feels Nation's Pain, Breasts" (classifies as not satire)

In [50]:
# Cell Block for showing your work with attempted headlines: correct

#classify.predict("Study Reveals: Babies are Dumb")
#classify.predict("World Death Rate Holding Steady at 100 percent")
#classify.predict("'No Way To Prevent This,' Says Only Nation Where This Regularly Happens")
#classify.predict("‘It’s Not Too Bad,’ Says Man As Hot Sauce Begins Disintegrating Jaw")
#classify.predict("Man Embraces Holiday Spirit By Telling People He Does not Speak To His Family")

#

#switch Ye for Man
classify.predict("Man off Twitter after praising hitler")

# ##################
#this one's almost 50/50
#classify.predict("Lucky Old Woman Getting Wheeled Around The Airport")
###############

#classify.predict("Man Feeling Pressure To Live Up To Conversation Between Barber And Customer In Next Chair")

#classify.predict("What White Voters See in Herschel Walker?")

# Cell Block for showing your work with attempted headlines: incorrect

#classify.predict("New President Feels Nation's Pain, Breasts")
#classify.predict("Kitten Thinks of Nothing but Murder All Day")
#classify.predict("Pop Culture Expert Surprisingly Not Ashamed of Self")
#classify.predict("Fun Toy Banned Because Of Three Stupid Dead Kids")
#classify.predict("Fuck Everything, We’re Doing Five Blades")

('1', tensor(1), tensor([0.1322, 0.8678]))

Now, we want to find two headlines that the classifier is really confident about, but classifies incorrectly. We want the confidence of the prediction to be at least 85%.

One headline is anything you want to write. Another must be a real headline (not satire) that you could trick the classifier into misclassifying changing only one word. For instance, taking `"Despair for Many and Silver Linings for Some in California Wildfires"`, a real NYTimes headline, you can change it to `"Despair for Many and Silver Linings for Some in Oregon Wildfires"` (note that this particular change does not cause the classifier to misclassify).

### Q4.3: One headline that the classifier classifies incorrectly, with false high confidence. (4% effort)

**ANSWER:** 
- "New President Feels Nation's Pain, Breasts"
    - classifies as non satire with a confidence of 93%

**### Q4.4: One real headline, with one word changed, that the classifier classifies as satire, with false high confidence. (4% effort)** !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

**ANSWER:**   
example of changing Kanye to Man -> confidence : 86% 
  - "Kanye off Twitter after praising hitler"
    - ('0', tensor(0), tensor([0.7098, 0.2902]))
  -  "Man off Twitter after praising hitler"
    - ('1', tensor(1), tensor([0.1322, 0.8678]))

src : 
https://www.cnn.com/videos/tv/2022/12/03/exp-tsr-todd-kanye-suspended-from-twitter.cnn

## Step 4b: What kinds of headlines are misclassified? (10% effort)

### Q4.5: Write your hypothesis below on what kinds of headlines are misclassified. 
Show your work. Try a bunch of examples and see what you can get to predict correctly/incorrectly.

(fastai v1 used to have a `TextClassificationInterpretation` utility that _might_ have been replaced with [this documentation on Interpretation of Predictions](https://docs.fast.ai/interpret.html) or _maybe_ [fastinference's text inference](https://muellerzr.github.io/fastinference/text.inference/#TextLearner.intrinsic_attention). Not required for answering this question, but could be useful?). 

**ANSWER:** 
- from the code below, here are my hypotheses :

  - from the fastai interpretation, in most of the misclassifications, what happens is that the satire conveyed "gets lost" when words are not recognized
  - a lot of the misclassified headlines are celebrity news and entertainment related
  - most of the headlines predicted as satire while being non-satire begin with "study proves/shows..."
  - Satirical Headlines ending in question marks (questions) are most often falsely predicted as not satire 
  - some headlines have more subtle satire, making them harder to predict if you're not a human user, such as "could hillary clinton have what it takes to defeat the democrats in 2008 ? "

In [56]:
# Show your work here
# I tried a bunch of examples in the cell above this one

interpret = Interpretation.from_learner(classify, ds_idx=0)
interpret.plot_top_losses(15)

0#xxbos could hillary clinton have what it takes to defeat the democrats in 2008 ?	1	0	0.9909366965293884	6.635965347290039
1	#xxbos they might be giants behind the music episode lacks sex , drugs	1	0	0.9942322373390198	6.220970630645752
2	#xxbos are we meeting the needs of our nation 's rich ?	1	0	0.9986876845359802	5.521677494049072
3	#xxbos the xxunk revolution : why are the media ignoring it ?	1	0	0.9980126619338989	5.387961387634277
4	#xxbos inside : the fetish photography of german chancellor xxunk xxunk	1	0	0.9954286813735962	5.3855695724487305
5	#xxbos study proves exactly how gross bathroom hand xxunk really are	0	1	0.9954178333282471	5.155484676361084
6	#xxbos the backstreet boys or ' n sync release new album	1	0	0.992737352848053	4.925014495849609
7	#xxbos study finds older dads may have ' xxunk ' sons	0	1	0.9900872111320496	4.801979064941406
8	#xxbos our nation 's celebrities : what are they wearing ?	1	0	0.9886727929115295	4.716418266296387
9	#xxbos the titanic scenario : could it really happen ?	1	0	0.9960008263587952	4.703517913818359
10#xxbos the media : are they media - obsessed ?	1	0	0.991786539554596	4.690388202667236
11#xxbos whatsapp finally adds fully - encrypted video calling service	0	1	0.9908168315887451	4.613931179046631
12#xxbos three of man 's closest relationships with brands	1	0	0.990044355392456	4.609612941741943
13#xxbos the elderly : do they suspect ?	1	0	0.9881520867347717	4.480550289154053
14#xxbos the onion apologizes	1	0	0.9910528063774109	4.435608863830566

,input,target,predicted,probability,loss
0,xxbos could hillary clinton have what it takes to defeat the democrats in 2008 ?,1,0,0.9909366965293884,6.635965347290039
1,"xxbos they might be giants behind the music episode lacks sex , drugs",1,0,0.9942322373390198,6.220970630645752
2,xxbos are we meeting the needs of our nation 's rich ?,1,0,0.9986876845359802,5.521677494049072
3,xxbos the xxunk revolution : why are the media ignoring it ?,1,0,0.9980126619338989,5.387961387634277
4,xxbos inside : the fetish photography of german chancellor xxunk xxunk,1,0,0.9954286813735962,5.3855695724487305
5,xxbos study proves exactly how gross bathroom hand xxunk really are,0,1,0.9954178333282471,5.155484676361084
6,xxbos the backstreet boys or ' n sync release new album,1,0,0.992737352848053,4.925014495849609
7,xxbos study finds older dads may have ' xxunk ' sons,0,1,0.9900872111320496,4.801979064941406
8,xxbos our nation 's celebrities : what are they wearing ?,1,0,0.9886727929115295,4.716418266296387
9,xxbos the titanic scenario : could it really happen ?,1,0,0.9960008263587952,4.703517913818359


14

# Step 5: Save your classifier
Now that we've trained the classifier, you're ready for Part 2. You'll use this saved file in your bot later.

In [57]:
classify.export(fname='satire_awd.pkl')

Later, you'll use it like so.

In [58]:
serve_classifier = load_learner(fname=str(data_path)+'/satire_awd.pkl')
serve_lm = load_learner(fname=str(data_path)+'/headlines-lm.pkl')

In [59]:
serve_classifier.predict('How the New Syria Took Shape')

('0', tensor(0), tensor([0.9765, 0.0235]))

In [60]:
serve_lm.predict('Rising Seas', n_words=7)

'Rising Xxunk for the fast food supply industry happened'

# Step 6: Set-up your Twitter Bot (20% effort)

Follow the instructions in Glow to set-up your Twitter Developer account! Twitter has been changing its Developer/Authentication processes lately, and it's hard to predict just exactly what we need to get everything to work - I suspect this includes applying for Elevated Developer Status as well. If you're successful at this section, let your instructor know as soon as possible!

_Remember_ if you can't get the Twitter/Tweepy API to work, it is possible to build a chat bot outside of Twitter, just in this notebook for your instructor to interact with! Don't leave this section incomplete!

### Q6.1: Reflect on the sign-up process (5% effort) 
Anytime you're giving out your info, you should think of who is going to have access to it. Who is going to read it. What is Twitter actually doing with it? 

As you go through the process of signing up for a Twitter developer account, reflect on the questions you are asked, why you are being asked them, and how you think they will serve the intended purpose (or not). Why might Twitter care if we are a government entity? Or displaying information outside of Twitter? 


**ANSWER:** 
- these are the questions that stuck out to me when setting up my twitter developer account :

- **Will your app use Tweet, Retweet, Like, Follow, or Direct Message functionality?** my understanding is that Twitter generates Access Tokens with different permissions (read  only, read and write, read and write and direct message), which allow other people to do actions on behalf of the original user with the account, so I think this question might be related to the kind of access token that will be generated later. 

- **Do you plan to display Tweets or aggregate data about Twitter content outside Twitter?** this is important because Twitter has to know where its data ends up, in the developer policy, it is stated that "You must maintain the integrity of all Twitter Content that you display publicly or to people who use your service". So if you say yes, you have to explain how/where you are going to display twitter data and see if twitter approves of it. For example, doxxing is one scenario where you would violate Twitter Developer policy by displaying twitter data outside twitter after collecting information about certain user(s).

- **Will your product, service, or analysis make Twitter content or derived information available to a government entity?** Twitter Policy prohibits the user of Twitter data for surveillance purposes, and that's the first purpose I think of when I think of government entities using Twitter data. Twitter developer policy also includes other examples of malicious use of data, such as investigating or tracking sensitive groups and organizations, such as unions or activist groups, Background checks or any form of extreme vetting, Credit or insurance risk analyses, Individual profiling or psychographic segmentation, and *Facial recognition* .

## Important! Tweet Rate Limit!
It's possible that Twitter will rate limit you developer account, try to reply/post sparingly during your testing phase! Maybe try building your bot _outside_ of Twitter before connecting it to Twitter, to ensure that you don't end up in an endless loop of Tweeting...

## (6.2) Test whether your script has access to Twitter
I'm assuming you've stored your Twitter keys in `credentials.py` saved in the same directory as this notebook, and the file looks like the following format:
```
consumer_key = 'XXX' # API key
consumer_secret = 'XXX' # API secret
access_token = 'XXX' # Access token
access_token_secret = 'XXX' # Access secret
# don't see the last two access tokens? You need to generate them after you create the bot (through the developer dashboard). 
```

Twitter/OAuth2.0/Tweepy have been changing how they provide access to the API. It _might_ be possible to do the below with just a Bearer Token (then again, it might not). You can try replacing the two `auth` lines below with `auth = tweepy.OAuth2BearerHandler(bearer_token_here)` (see [Tweepy OAuth documentation](https://docs.tweepy.org/en/stable/authentication.html))

The below snippet of code should access your Twitter account, fetch your name stored there, and then print it out. If it doesn’t, go back and debug!

In [90]:
import sys
import os
import tweepy
py_file_location = "/content/drive/My Drive/data"
sys.path.append(os.path.abspath(py_file_location))
from credentials import *


auth = tweepy.OAuthHandler("N5dsnExnOMBEODb6LvwrHch7o", "Fz0bPjG6s6oF7sk3DbPLsFMgEBTt2iK0xfAlNlp6ZsxlhhWu2D")
auth.set_access_token("1592532616535855105-LjRXnxQRtF8Hc93c9Bwk5iscXbGkwe", "oIHn3Z8oqjx3YvpN54UXQ5hPBQxuGLoJ8hap917ZjV3nw")
#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

user = api.get_settings()['screen_name']
print (user)
# Note: The name that's printed is the name of your bot.
#sassyybot


sassyybot


The next step is to find a tweet that mentioned your bot (starting with the `@` symbol), which are the messages that your bot will respond to. You'll need to tweet yoru bot for this to work! Here’s some template code for searching for tweets with Tweepy for a hypothetical bot, `@satirebot`, you'll need to modify it to fit your needs:

In [94]:
####
# Define the search
#####

query = '@sassyybot' # replace with your bot's twitter name!
max_tweets = 1000

####
# Do the search
#####
searched_tweets = []
last_id = -1 

while len(searched_tweets) < max_tweets:
    count = max_tweets - len(searched_tweets)
    try:
        # if using older version of Tweepy, you'll want to use api.search instead of api.search_tweets
        new_tweets = api.search(q=query, count=count, max_id=str(last_id - 1))
        if not new_tweets:
            break
        searched_tweets.extend(new_tweets)
        last_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # depending on TweepError.code, one may want to retry or wait
        # to keep things simple, we will give up on an error
        break
####
# Iterate over the search
#####
for status in searched_tweets:
    # do something with all these tweets
    print(status)

Status(_api=<tweepy.api.API object at 0x7f071d7c5d00>, _json={'created_at': 'Sat Dec 03 20:50:56 +0000 2022', 'id': 1599144173797769218, 'id_str': '1599144173797769218', 'text': '@sassyybot Key Talking Points From the US vs Netherlands Game', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'sassyybot', 'name': 'Satire bot', 'id': 1592532616535855105, 'id_str': '1592532616535855105', 'indices': [0, 10]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': 1592532616535855105, 'in_reply_to_user_id_str': '1592532616535855105', 'in_reply_to_screen_name': 'sassyybot', 'user': {'id': 1598868102019776512, 'id_str': '1598868102019776512', 'name': 'testing account', 'screen_name': 'testing21541122', 'location': '', 'description': '', 'u

If you run this snippet, it will print out the messy JSON version of all the statuses returned. You can see what sorts of things each of the status objects contain. Some important ones are:

* `text` -- the text of the returned tweet
* `author.screen_name` - the Twitter username of the user who sent the tweet
* `id` - id of the tweet, which you can use to reply to it, or search only for tweets that were posted after it

In [88]:
# Insert code in this cell to access the useful information from the JSON status:

#Useful information I found

 : _json={'created_at': 'Sat Dec 03 02:36:33 +0000 2022', 'id': 1598868762366615552,
                            'id_str': '1598868762366615552', 'text': '@sassyybot hello how are you pls work', 
                            'entities': {'hashtags': [], 'symbols': [], 'user_mentions': 
                            [{'screen_name': 'sassyybot', 'name': 'Satire bot', 'id': 1592532616535855105,
                            'id_str': '1592532616535855105',  
                            'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 
                            'in_reply_to_screen_name': 'sassyybot', 
                            'user': {'id': 1598868102019776512, 
                            'id_str': '1598868102019776512', 
                             'name': 'testing account', 'screen_name': 'testing21541122', 
                             'followers_count': 0, 'friends_count': 10, 'listed_count': 0, 
                              'created_at': 'Sat Dec 03 02:34:04 +0000 2022'}}}}
# THE WHOLE THING
 _json={'created_at': 'Sat Dec 03 02:36:33 +0000 2022', 'id': 1598868762366615552, 'id_str': '1598868762366615552', 'text': '@sassyybot hello how are you pls work', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'sassyybot', 'name': 'Satire bot', 'id': 1592532616535855105, 'id_str': '1592532616535855105', 'indices': [0, 10]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': 1592532616535855105, 'in_reply_to_user_id_str': '1592532616535855105', 'in_reply_to_screen_name': 'sassyybot', 'user': {'id': 1598868102019776512, 'id_str': '1598868102019776512', 'name': 'testing account', 'screen_name': 'testing21541122', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 0, 'friends_count': 10, 'listed_count': 0, 'created_at': 'Sat Dec 03 02:34:04 +0000 2022', 'favourites_count': 0, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 1, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': None, 'profile_background_image_url_https': None, 'profile_background_tile': False, 'profile_image_url': 'http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', 'profile_image_url_https': 'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', 'profile_link_color': '1DA1F2', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_text_color': '333333', 'profile_use_background_image': True, 'has_extended_profile': True, 'default_profile': True, 'default_profile_image': True, 'following': False, 'follow_request_sent': False, 'notifications': False, 'translator_type': 'none', 'withheld_in_countries': []}, 'geo': None, 'coordinates': None, 'place': None, 'contributors': None, 'is_quote_status': False, 'retweet_count': 0, 'favorite_count': 0, 'favorited': False, 'retweeted': False, 'lang': 'en'}, created_at=datetime.datetime(2022, 12, 3, 2, 36, 33), id=1598868762366615552, id_str='1598868762366615552', text='@sassyybot hello how are you pls work', truncated=False, entities={'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'sassyybot', 'name': 'Satire bot', 'id': 1592532616535855105, 'id_str': '1592532616535855105', 'indices': [0, 10]}], 'urls': []}, metadata={'iso_language_code': 'en', 'result_type': 'recent'}, source='Twitter for iPhone', source_url='http://twitter.com/download/iphone', in_reply_to_status_id=None, in_reply_to_status_id_str=None, in_reply_to_user_id=1592532616535855105, in_reply_to_user_id_str='1592532616535855105', in_reply_to_screen_name='sassyybot', author=User(_api=<tweepy.api.API object at 0x7fac6957fee0>, _json={'id': 1598868102019776512, 'id_str': '1598868102019776512', 'name': 'testing account', 'screen_name': 'testing21541122', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 0, 'friends_count': 10, 'listed_count': 0, 'created_at': 'Sat Dec 03 02:34:04 +0000 2022', 'favourites_count': 0, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 1, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': None, 'profile_background_image_url_https': None, 'profile_background_tile': False, 'profile_image_url': 'http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', 'profile_image_url_https': 'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', 'profile_link_color': '1DA1F2', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_text_color': '333333', 'profile_use_background_image': True, 'has_extended_profile': True, 'default_profile': True, 'default_profile_image': True, 'following': False, 'follow_request_sent': False, 'notifications': False, 'translator_type': 'none', 'withheld_in_countries': []}, id=1598868102019776512, id_str='1598868102019776512', name='testing account', screen_name='testing21541122', location='', description='', url=None, entities={'description': {'urls': []}}, protected=False, followers_count=0, friends_count=10, listed_count=0, created_at=datetime.datetime(2022, 12, 3, 2, 34, 4), favourites_count=0, utc_offset=None, time_zone=None, geo_enabled=False, verified=False, statuses_count=1, lang=None, contributors_enabled=False, is_translator=False, is_translation_enabled=False, profile_background_color='F5F8FA', profile_background_image_url=None, profile_background_image_url_https=None, profile_background_tile=False, profile_image_url='http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', profile_image_url_https='https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', profile_link_color='1DA1F2', profile_sidebar_border_color='C0DEED', profile_sidebar_fill_color='DDEEF6', profile_text_color='333333', profile_use_background_image=True, has_extended_profile=True, default_profile=True, default_profile_image=True, following=False, follow_request_sent=False, notifications=False, translator_type='none', withheld_in_countries=[]), user=User(_api=<tweepy.api.API object at 0x7fac6957fee0>, _json={'id': 1598868102019776512, 'id_str': '1598868102019776512', 'name': 'testing account', 'screen_name': 'testing21541122', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 0, 'friends_count': 10, 'listed_count': 0, 'created_at': 'Sat Dec 03 02:34:04 +0000 2022', 'favourites_count': 0, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 1, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': None, 'profile_background_image_url_https': None, 'profile_background_tile': False, 'profile_image_url': 'http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', 'profile_image_url_https': 'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', 'profile_link_color': '1DA1F2', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_text_color': '333333', 'profile_use_background_image': True, 'has_extended_profile': True, 'default_profile': True, 'default_profile_image': True, 'following': False, 'follow_request_sent': False, 'notifications': False, 'translator_type': 'none', 'withheld_in_countries': []}, id=1598868102019776512, id_str='1598868102019776512', name='testing account', screen_name='testing21541122', location='', description='', url=None, entities={'description': {'urls': []}}, protected=False, followers_count=0, friends_count=10, listed_count=0, created_at=datetime.datetime(2022, 12, 3, 2, 34, 4), favourites_count=0, utc_offset=None, time_zone=None, geo_enabled=False, verified=False, statuses_count=1, lang=None, contributors_enabled=False, is_translator=False, is_translation_enabled=False, profile_background_color='F5F8FA', profile_background_image_url=None, profile_background_image_url_https=None, profile_background_tile=False, profile_image_url='http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', profile_image_url_https='https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', profile_link_color='1DA1F2', profile_sidebar_border_color='C0DEED', profile_sidebar_fill_color='DDEEF6', profile_text_color='333333', profile_use_background_image=True, has_extended_profile=True, default_profile=True, default_profile_image=True, following=False, follow_request_sent=False, notifications=False, translator_type='none', withheld_in_countries=[]), geo=None, coordinates=None, place=None, contributors=None, is_quote_status=False, retweet_count=0, favorite_count=0, favorited=False, retweeted=False, lang='en')
           

IndentationError: ignored

And, of course, you want to be able to send new tweets too, which is done like this:

In [95]:
api.update_status('I\'m in a plane!')

TweepError: ignored

You will likely want to not just send a status, but actually reply to the original tweet. You can combine some of what you’ve learned here to do that (You have to include the @-mention in the tweet, or it won’t show up as a reply.):

In [93]:
# if using older version of Tweepy, you don't want/need keyword arguments, like:
#api.update_status('this is a reply! @' + status.author.screen_name, status.id_str)

#text = 'this is a reply! @' + 'testing21541122'
#api.update_status(status = "no",  in_reply_to_status_id ='1598868762366615552')

text = 'this is a reply! @' + 'testing21541122'
api.update_status(status = 'testing again', in_reply_to_status_id ='1598868762366615552')

#api.update_status('no! @' + 'testing21541122', '1598868762366615552')

TweepError: ignored

### Q6.2: Adapt the code from above to respond to a query, as described above. (15% effort)

Copy the code from above and modify it to search for Tweets referencing your bot (i.e., Tweets containing `@YourBotName`), and then reply to those tweets, perhaps with something using the tweet's text such as the number of characters in the original tweet.

Of course, this is just the tip of the iceberg with what you can do with Tweepy. If you’re interested, there is a lot of documentation online. You might start with the [official Tweepy docs](http://docs.tweepy.org/en/v3.6.0/getting_started.html).

In [ ]:
# Insert code here

####
# Define the search
#####

query = '@sassyybot' # replace with your bot's twitter name!
max_tweets = 100

####
# Do the search
#####
searched_tweets = []
last_id = -1 

while len(searched_tweets) < max_tweets:
    count = max_tweets - len(searched_tweets)
    try:
        # if using older version of Tweepy, you'll want to use api.search instead of api.search_tweets
        new_tweets = api.search(q=query, count=count, max_id=str(last_id - 1))
        if not new_tweets:
            break
        searched_tweets.extend(new_tweets)
        last_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # depending on TweepError.code, one may want to retry or wait
        # to keep things simple, we will give up on an error
        break
####
# Iterate over the search
#####
for status in searched_tweets:
    texts = '@' + status.author.screen_name + ' your tweet has ' + str(len(status.text)) + ' words! '
    api.update_status(status = texts,  in_reply_to_status_id = status.id_str)
    print(status)

TweepError: ignored

# Step 7: Integrate the bot with the satire classifier (40% effort)
Now that you can do basic replies with your bot, it’s time to make it do something useful! Specifically, our bot should do two things: 

1. When someone tweets a headline @ the bot, it replies with whether the headline is satire. 
2. It also makes up a headline that plays off the original headline, and tweets it back. 

Here’s an example (assuming the bot’s called @satirebot):

**User: @satirebot Rising Seas Will Erase More Cities by 2050, New Research Shows**

**satirebot: Yeah, looks real, not satire. But here’s what I say: “Rising Seas will sound great on national security”**


_Hint_: you want to call `api.update_status` using the outputs of our models (classifier and language models) instead of responding `this is a reply!`

The best assignments will have bots that:

- Respond with prediction of whether the given headline is satire or not (10% effort)
- Respond with a newly generated headline inspired by the original (10% effort)
- Interactions are designed to be friendly, non-misleading, and should let users to discover what is going on. (10% effort)

In [85]:
# Feel free to add more cell blocks, as needed

import random
import re 
import fastai
#from fastai.vision.all import *
from fastai import *
from fastai.learner import load_learner
#from fastai import load_learner
####
# Define the search
#####

#serve_classifier.predict
serve_classifier = load_learner(fname=str(data_path)+'/satire_awd.pkl')
serve_lm = load_learner(fname=str(data_path)+'/headlines-lm.pkl')

query = '@sassyybot' # replace with your bot's twitter name!
max_tweets = 100

####
# Do the search
#####
searched_tweets = []
last_id = -1 

while len(searched_tweets) < max_tweets:
    count = max_tweets - len(searched_tweets)
    try:
        # if using older version of Tweepy, you'll want to use api.search instead of api.search_tweets
        new_tweets = api.search(q=query, count=count, max_id=str(last_id - 1))
        if not new_tweets:
            break
        searched_tweets.extend(new_tweets)
        last_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # depending on TweepError.code, one may want to retry or wait
        # to keep things simple, we will give up on an error
        break
####
# Iterate over the search
#####

## generating first part of the tweet reply : is the tweet in question satirical or not?
def isSatire(status):
  satire = serve_classifier.predict(status.text)
  if(satire[0] == '0'):
      #three options for responding to each of the satirical/non-satirical tweets
      text_satire1 = "well, this one doesn't look like satire to me"
      text_satire2 =  "yeah this one looks real. Not satire"
      text_satire3 = "As far as I'm concerned, this one's real. Not satire"
      text = (random.choice([text_satire1, text_satire2, text_satire3]))
  if(satire[0] == '1'):
      text_notsatire1 = "Yes, this one's satire"
      text_notsatire2 = "I see you're being Satirical."
      text_notsatire3 =  "looks like someone's in the mood for satire."
      text = random.choice([text_notsatire1, text_notsatire2, text_notsatire3])
  return text

def unknownWord(tweet, tweetreply):
  tweetreply = tweetreply.split()
  for i in range(len(tweet)):
    if( (tweetreply[i] == "Xxunk") | (tweetreply[i]== "xxunk")):
      tweetreply[i] = tweet[i]
  return tweetreply


for status in searched_tweets:  
    #for empty tweets, do nothing
    if status.text.strip() == "":
        continue
 
    words = status.text.split()
    tweetwordcount = len(words)
    textreply = isSatire(status)
    replystart = status.text.split()[1:6]
    print(replystart)

    # base tweet reply on the first few words of the original tweet and generate the rest
    replystartlen = len(replystart)
    textreply2 = serve_lm.predict(replystart, n_words=tweetwordcount - (replystartlen-1))
     
     #handling uknown words by replacing them with the tweet's original words
    textreply2 = unknownWord(replystart, textreply2)

    print(textreply2)
    while(not serve_classifier.predict(textreply2)):
        textreply2 = serve_lm.predict(replystart, n_words=tweetwordcount - (replystartlen-1))
        textreply2 = unknownWord(replystart, textreply2)
        if(serve_lm.predict(textreply2)):
            break
    textreply2 = ''.join(textreply2)
    texts = '@' + status.author.screen_name + ' ' + textreply + '. But here is what I say : ' + textreply2 
    api.update_status(status = texts,  in_reply_to_status_id = status.id_str) 
    print(status)


    #stuff it generated that left me speechless 
        #study shows that babies have never looked properly for sexual gratification
        # omg it worked chubby 
        # omg it worked out so hard
        #study shows that babies do n't become sexually active until coming 
        #study shows that babies are burning like all tiny burned outside
        #study shows that babies are a fat little princess

       # --> maybe that's why my account got restricted ;-;

['Key', 'Talking', 'Points', 'From', 'the']


['Key', 'Talking', 'Points', 'From', 'the', 'annual', 'temperature', 'chain', 'climb', 'with', 'a']


TweepError: ignored

In [ ]:
"from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Q7.1: Reflect: Would you recommend using our satire-classifier as a good starting point to build a fake-news classifier? (10% effort)
If so, what changes would we need to make to make it useful for this purpose? If not, why not?

**ANSWER:** 

I think it would make a good starting point for a fake-news classifier, because the general idea of a bot that can both generate satirical headlines and detect them could be applied to make the fake news detection bot.

I was thinking we could have the teacher-learner model where the learner will generate fake stories that are realistic to read for both human users as well as the teacher. Then, the teacher will classify news stories as real or fake. However, we would need to train it on a lot more data, as well as find a way to incorporate the patterns of how news articles spread when they’re fake vs real. For example, where they first appear, the author’s number of citations, how quickly they went viral, and on which platform(s), etc...

We can use mechanical Turk to help train and evaluate the accuracy of our algorithm. This approach, however, has some challenges. Even humans sometimes can’t tell when an article is fake, because it’s harder in this context to find a label or a specific indicator that can tell us that the news is fake. Whereas in the satire bot, the use of superlatives, ‘man’, and other words are pretty good indicators of the headline being satirical. Also, the bot’s potential to generate convincing fake news is concerning, especially for people who don’t have access to the bot, the fake news detection bot could end up contributing to the spread of misinformation instead of combatting it.

# Extra Credit 1: Test with Users and Iterate (5% additional effort)
In this part, you’ll ask three participants to interact with your bot. You’ll give the user high-level information about what the domain of the bot is, and then see how they interact with it. Ask each of the participants to ask your chatbot at least three different things. Record how they interact with your bot. After this participant input, update your bot to attempt to address how that participant interacted with your chatbot. 

### Q EC1.1: How did what your participants input compare to the ones you tested so far? How did participants react when the chatbot didn’t respond correctly, or responded with nonsense? (2.5% additional effort)

**ANSWER:** _Double click this text to write your answer to the question here._

### Q EC1.2: What change could you make in response to this feedback? (2.5% additional effort)

**ANSWER:** _Double click this text to write your answer to the question here._

# Extra Credit 2: Deploy bot (10% additional effort)
Take this Jupyter notebook and deploy it such that it runs once an hour, and responds to all messages sent to it  
([Tweepy StreamListener](https://docs.tweepy.org/en/v3.10.0/streaming_how_to.html) might be useful here). You do not have to worry about finding a server to deploy this independently, but it should be runnable from the cell below. Run the bot, and then force-stop it so that I can see the output.

### Q EC2.1: Twitter handle (of bot) to test for.

**ANSWER:** _Double click this text to write your answer to the question here._

In [ ]:
# Feel free to add more cell blocks, as needed

# Assignment Submission

Once you've completed all of the above, you're done with assignment 4! As always, clean up your code and ensure your entire Jupyter Notebook runs before submitting, Iris must be able to run your notebook on her machine.

Once you think everything is set, please change the filename of your notebook to `[yourunixID]_haiiYY[assignmentnumber].ipynb`, e.g., `ikh1_haii17a4.ipynb` and then .zip your Notebook and any additional files you used (this is likely just the Notebook file and your JSON data in the lib/ directory: **DON'T** include your gigantic .pkl language model files, and **DON'T** include your credentials.py!), and submit the `.zip` file on GLOW.